In [2]:
print("abc"+"\0"+"123")

abc 123


In [3]:
import struct
message_bit_length=64
struct.pack(b'>Q', message_bit_length)

b'\x00\x00\x00\x00\x00\x00\x00@'

In [4]:
url = "https://cse207b.nh.cryptanalysis.fun/hw4/api?token=17001485f42284b8c6775983fe83eaa1651e0d54&user=admin&get_file=kitten.jpg"

In [5]:
question_mark_i = url.find('?')
url_without_param = url[:question_mark_i]
param_str = url[question_mark_i+1:]
first_and_i = param_str.find("&")
token_param_str = param_str[:first_and_i]
rest_of_q_str = param_str[first_and_i+1:]
assert token_param_str.startswith("token=")

token_hex_str = token_param_str[6:]
hash_hidden_states=[0]*5
for i in range(len(hash_hidden_states)):
    hash_hidden_states[i] = int(token_hex_str[8*i:8*i+8],16)

In [6]:
hash_hidden_states[0]

385881221

In [7]:
import sha1

In [8]:
key_byte_cnt=12

In [9]:
key_and_q_str = "1"*key_byte_cnt+rest_of_q_str

In [10]:
hash_output,processed_chunks=sha1.mal1(key_and_q_str.encode(encoding="ascii"))

In [11]:
def to_url_char(byte_value:int)->str:
    try_result = chr(byte_value)
    if try_result in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-._~:/?#[]@!$&'()*+,;=":
        return try_result
    return "%"+("%02x"%byte_value)

In [22]:
processed_chunks

[b'111111111111user=admin&get_file=kitten.jpg\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01P']

In [13]:
param_str_constructed=""
for p_chunk in processed_chunks:
    print(len(p_chunk))

64


In [19]:
param_string_to_append = "&get_file=hw4.pdf"
p_str_to_append_bytes = param_string_to_append.encode("ascii")
last_chunk = p_str_to_append_bytes
total_size_in_byte = 64*len(processed_chunks)+len(param_string_to_append)
last_chunk += b'\x80'
last_chunk += b'\x00' * ((56 - (total_size_in_byte + 1) % 64) % 64)
total_size_in_bit = 8*total_size_in_byte
last_chunk += struct.pack(b'>Q', total_size_in_bit)
h = sha1._process_chunk(last_chunk[:64],*hash_hidden_states)
if len(last_chunk)>64:
    h = sha1._process_chunk(last_chunk[64:],*h)
hash_hex_str = ""
for hash_hidden_state_int64_num in h:
    hash_hex_str+=("%08x"%hash_hidden_state_int64_num)

In [30]:
query_string_crafted = ""
for p_chunk in processed_chunks:
    for byte1 in p_chunk:
        query_string_crafted+=to_url_char(byte1)
query_string_crafted+=param_string_to_append
query_string_crafted = query_string_crafted[key_byte_cnt:]

In [32]:
res = url_without_param+"?"+"token="+hash_hex_str+"&"+query_string_crafted

In [33]:
res

'https://cse207b.nh.cryptanalysis.fun/hw4/api?token=6d5d20c60ac0e6e37419c13ddd1ca5f0825f7bdc&user=admin&get_file=kitten.jpg%80%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%01P&get_file=hw4.pdf'

In [ ]:
import requests

play_url = "https://cse207b.nh.cryptanalysis.fun/hw4/api?token=17001485f42284b8c6775983fe83eaa1651e0d54&user=admin&get_file=kitten.jpg"
response = requests.get(play_url)

In [ ]:
len(response.content)

560990